In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import pickle
import itertools
import math
# from scipy.stats import ranksums
# from collections import Counter
import glob
import ast

In [2]:
top500genes = open("/home/vivek/jupyter_notebooks/bio_dgi_extension/ppi_string_merged/new_model4/best_performing_model/top_500_genes.txt").read().split("\n")
top500genes = [i.split("\t")[0] for i in top500genes]
gene798 = open("/home/vivek/jupyter_notebooks/bio_dgi_extension/all_genes_798.txt").read().split('\n')
gene798 = [i for i in gene798 if i]
gene34 = open("/home/vivek/jupyter_notebooks/bio_dgi_extension/farcastbio/genelist77_mm_mgus.txt").read().split("\n")
gene34 = [i for i in gene34 if i]
gene77 = open("/home/vivek/jupyter_notebooks/bio_dgi_extension/farcastbio/77genelist.txt").read().split("\n")
gene77 = [i for i in gene77 if i]

mmrf_sample_list = open('/home/vivek/mmrf_data/mmrf_variant_callers/mmrf_mm_sample_list.txt').read().split('\n')
mmrf_sample_list = [x for x in mmrf_sample_list if x != '']
mmrf_sample_list = [i for i in mmrf_sample_list if not "PB" in i]
aiims_mm_sample_list = open('/home/vivek/aiims_data_processing/scripts/mm_list.txt').read().split('\n')
aiims_mm_sample_list = [x for x in aiims_mm_sample_list if x != '']
aiims_mgus_sample_list = open('/home/vivek/aiims_data_processing/scripts/mgus_list.txt').read().split('\n')
aiims_mgus_sample_list = [x for x in aiims_mgus_sample_list if x != '']
ega_sample_list = open('/home/vivek/ega_data_1901/scripts/ega_mgus_list.txt').read().split('\n')
ega_sample_list = [x for x in ega_sample_list if x != '']
all_mm = list(set(mmrf_sample_list + aiims_mm_sample_list))
all_mgus = list(set(aiims_mgus_sample_list + ega_sample_list))
# all_mm = [i for i in all_mm if not "_2_BM_" in i]
# all_mm = [i for i in all_mm if not "_3_BM_" in i]
# all_mm = [i for i in all_mm if not "_4_BM_" in i]

mmrf_muse_path = "/home/vivek/mmrf_data/mmrf_variant_callers/hg19/muse/annovar/vcf_with_AF/vcf_reheader/fathmm/filtered_vcf/"
mmrf_mutect_path = "/home/vivek/mmrf_data/mmrf_variant_callers/hg19/mutect2/annovar/vcf_reheader/fathmm/filtered_vcf"
mmrf_sniper_path = "/home/vivek/mmrf_data/mmrf_variant_callers/hg19/somatic_sniper/annovar/vcf_with_AF/vcf_reheader/fathmm/filtered_vcf"
mmrf_varscan_path = "/home/vivek/mmrf_data/mmrf_variant_callers/hg19/varscan2/annovar/vcf_reheader/fathmm/filtered_vcf"

ega_muse_path = "/home/vivek/ega_data_1901/muse/fathmm_filtered/filtered_vcf"
ega_mutect_path = "/home/vivek/ega_data_1901/MUTECT2_analysis/fathmm_filtered/filtered_vcf"
ega_sniper_path = "/home/vivek/ega_data_1901/SNIPER_analysis/fathmm_filtered/filtered_vcf"
ega_varscan_path = "/home/vivek/ega_data_1901/varscan2/fathmm_filtered/filtered_vcf"

aiims_mm_muse_path = "/home/vivek/aiims_data_processing/muse/mm_sample_analysis/fathmm_filtered/filtered_vcf"
aiims_mm_mutect_path = "/home/vivek/aiims_data_processing/MUTECT2/mm_sample_analysis/fathmm_filtered/filtered_vcf"
aiims_mm_sniper_path = "/home/vivek/aiims_data_processing/SNIPER/mm_sample_analysis/fathmm_filtered/filtered_vcf"
aiims_mm_varscan_path = "/home/vivek/aiims_data_processing/varscan2/mm_annovar/vcf_reheader/fathmm/filtered_vcf"

aiims_mgus_muse_path = "/home/vivek/aiims_data_processing/muse/mgus_sample_analysis/fathmm_filtered/filtered_vcf"
aiims_mgus_mutect_path = "/home/vivek/aiims_data_processing/MUTECT2/mgus_sample_analysis/fathmm_filtered/filtered_vcf"
aiims_mgus_sniper_path = "/home/vivek/aiims_data_processing/SNIPER/mgus_sample_analysis/fathmm_filtered/filtered_vcf"
aiims_mgus_varscan_path = "/home/vivek/aiims_data_processing/varscan2/mgus_annovar/vcf_reheader/fathmm/filtered_vcf"

gene_variants = pd.read_excel("/home/vivek/jupyter_notebooks/bio_dgi_extension/final_compilation_work/lof/lof798_ghis.xlsx")

aiims_mgus_cnv_path = "/home/vivek/aiims_data_processing/cnvkit_analysis/mgus"
aiims_mm_cnv_path = "/home/vivek/aiims_data_processing/cnvkit_analysis/mm"
ega_cnv_path = "/home/vivek/ega_data_1901/cnvkit_analysis"
mmrf_cnv_path = "/home/vivek/mmrf_data/cnv_analysis/filtered_cns"

# with open('/home/vivek/jupyter_notebooks/bio_dgi_extension/cnv_analysis/mm_dict.pkl', 'rb') as f:
#     mm_cnv_data = pickle.load(f)
    
# with open('/home/vivek/jupyter_notebooks/bio_dgi_extension/cnv_analysis/mgus_dict.pkl', 'rb') as f:
#     mgus_cnv_data = pickle.load(f)
    
gene_katz_centrality = pd.read_excel("../gene_centrality/katz_centrality.xlsx", index_col=0)

antigen_pathway = open('/home/vivek/jupyter_notebooks/bio_dgi_extension/farcastbio/five_pathways/antigen presentation pathway.txt').read().split('\n')
antigen_pathway = [x for x in antigen_pathway if x != '']
cyto_pathway = open('/home/vivek/jupyter_notebooks/bio_dgi_extension/farcastbio/five_pathways/cytoskeleton-rearrangement-genes.txt').read().split('\n')
cyto_pathway = [x for x in cyto_pathway if x != '']
histone_pathway = open("/home/vivek/jupyter_notebooks/bio_dgi_extension/farcastbio/five_pathways/histone-modification-genes.txt").read().split('\n')
histone_pathway = [x for x in histone_pathway if x != '']
myeloid_pathway = open('/home/vivek/jupyter_notebooks/bio_dgi_extension/farcastbio/five_pathways/myeloid-checkpoints-genes.txt').read().split('\n')
myelois_pathway = [x for x in myeloid_pathway if x != '']
nk_pathway = open('/home/vivek/jupyter_notebooks/bio_dgi_extension/farcastbio/five_pathways/NK-Cell Pathway Genes.txt').read().split('\n')
nk_pathway = [x for x in nk_pathway if x != '']

driver_list1 = pd.read_csv('/home/vivek/driverGenes_Oncogene_TSG/IntOGen-DriverGenes_MM.tsv', sep = '\t', index_col = 0).index.tolist()
driver_list1 = [i.replace(' ','') for i in driver_list1]
driver_list1 = [i for i in driver_list1 if i]
driver_list2 = open('/home/vivek/driverGenes_Oncogene_TSG/list_of_63_driver_genes.txt').read().split('\n')
driver_list2 = [i.replace(' ','') for i in driver_list2]
driver_list2 = [i for i in driver_list2 if i]
driver_list3 = open('/home/vivek/driverGenes_Oncogene_TSG/Genomic_landscape_and_chronological_reconstruction_of_driver_events_in_multiple.txt').read().split('\n')
driver_list3 = [i.replace(' ','') for i in driver_list3]
driver_list3 = [i for i in driver_list3 if i]
driver_list4 = open("/home/vivek/driverGenes_Oncogene_TSG/driver_literature/heterogeneity_of_genomic_evolution_and_mutational_profiles_mm.txt").read().split('\n')
driver_list4 = [i.replace(' ','') for i in driver_list4]
driver_list4 = [i for i in driver_list4 if i]
driver_list5 = open("/home/vivek/driverGenes_Oncogene_TSG/driver_literature/revealing_impact_of_sv.txt").read().split('\n')
driver_list5 = [i.replace(' ','') for i in driver_list5]
driver_list5 = [i for i in driver_list5 if i]
driver_list6 = open("/home/vivek/driverGenes_Oncogene_TSG/driver_literature/role_of_aid.txt").read().split('\n')
driver_list6 = [i.replace(' ','') for i in driver_list6]
driver_list6 = [i for i in driver_list6 if i]
driver_list7 = open("/home/vivek/driverGenes_Oncogene_TSG/driver_literature/wgs_of_mm_reveals_oncogenic_pathways_are_targeted.txt").read().split('\n')
driver_list7 = [i.replace(' ','') for i in driver_list7]
driver_list7 = [i for i in driver_list7 if i]
driver_list = list(set(driver_list1 + driver_list2 + driver_list3 + driver_list4 + driver_list5 + driver_list6 + driver_list7))

mm_unique_genes = open("../../mm_798.txt").read().split("\n")
mm_unique_genes = [i for i in mm_unique_genes if i]
mgus_unique_genes = open("../../mgus_798.txt").read().split("\n")
mgus_unique_genes = [i for i in mgus_unique_genes if i]

"""
    Gene Categories:
    ----------------
    Category-A: Gene that are significantly altered in MM and not in MGUS. [Transformative genes]
    Category-B: Gene that are significantly altered in MGUS and not in MM. [Rejected genes]
    Category-C: Gene that are significantly altered in both MM and MGUS. [disease initiating genes]
"""

cat_a = [i for i in mm_unique_genes if i not in mgus_unique_genes]
cat_b = [i for i in mgus_unique_genes if i not in mm_unique_genes]
cat_c = [i for i in mgus_unique_genes if i in mm_unique_genes]

In [3]:
def gene_centrality(gen_name):
    try:
        centrality_score = gene_katz_centrality[gene_katz_centrality.index == gen_name]['katz_centrality'].values[0]
        comm = gene_katz_centrality[gene_katz_centrality.index == gen_name]['community'].values[0]
        # return '/'.join([str(comm), str(centrality_score)])
        return centrality_score, comm
    except:
        return np.nan, np.nan

In [4]:
def get_gene_df(sampleid, disease, gene_name):
    
    if "MMRF" in sampleid:
        df_muse = pd.read_csv(os.path.join(mmrf_muse_path, sampleid + ".vcf"), sep="\t")
        df_mutect = pd.read_csv(os.path.join(mmrf_mutect_path, sampleid + ".vcf"), sep="\t")
        df_sniper = pd.read_csv(os.path.join(mmrf_sniper_path, sampleid + ".vcf"), sep="\t")
        df_varscan = pd.read_csv(os.path.join(mmrf_varscan_path, sampleid + ".vcf"), sep="\t")
        df = pd.concat([df_muse, df_mutect, df_sniper, df_varscan])
        df = df[df["Gene_refGene"] == gene_name]
        df = df.drop_duplicates(ignore_index=True)
        
    if "CR" in sampleid:
        df_muse = pd.read_csv(os.path.join(ega_muse_path, sampleid + ".vcf"), sep="\t")
        df_mutect = pd.read_csv(os.path.join(ega_mutect_path, sampleid + ".vcf"), sep="\t")
        df_sniper = pd.read_csv(os.path.join(ega_sniper_path, sampleid + ".vcf"), sep="\t")
        df_varscan = pd.read_csv(os.path.join(ega_varscan_path, sampleid + ".vcf"), sep="\t")
        df = pd.concat([df_muse, df_mutect, df_sniper, df_varscan])
        df = df[df["Gene_refGene"] == gene_name]
        df = df.drop_duplicates()    
    
    if "SM" in sampleid:
        if disease == "mm":
            df_muse = pd.read_csv(os.path.join(aiims_mm_muse_path, sampleid + ".vcf"), sep="\t")
            df_mutect = pd.read_csv(os.path.join(aiims_mm_mutect_path, sampleid + ".vcf"), sep="\t")
            df_sniper = pd.read_csv(os.path.join(aiims_mm_sniper_path, sampleid + ".vcf"), sep="\t")
            df_varscan = pd.read_csv(os.path.join(aiims_mm_varscan_path, sampleid + ".vcf"), sep="\t")
            df = pd.concat([df_muse, df_mutect, df_sniper, df_varscan])
            df = df[df["Gene_refGene"] == gene_name]
            df = df.drop_duplicates()
        elif disease == "mgus":
            df_muse = pd.read_csv(os.path.join(aiims_mgus_muse_path, sampleid + ".vcf"), sep="\t")
            df_mutect = pd.read_csv(os.path.join(aiims_mgus_mutect_path, sampleid + ".vcf"), sep="\t")
            df_sniper = pd.read_csv(os.path.join(aiims_mgus_sniper_path, sampleid + ".vcf"), sep="\t")
            df_varscan = pd.read_csv(os.path.join(aiims_mgus_varscan_path, sampleid + ".vcf"), sep="\t")
            df = pd.concat([df_muse, df_mutect, df_sniper, df_varscan])
            df = df[df["Gene_refGene"] == gene_name]
            df = df.drop_duplicates()
        
    df['sample_id'] = [sampleid for i in range(len(df))]
    
    return df


def get_gene_snvs_all_samples(gene_name, disease):
    
    mm_sample_count, mgus_sample_count = 0,0
    mm_samplelevel_count, mgus_samplelevel_count = [], []
    if disease == "mm":
        for sam in all_mm:
            df = get_gene_df(sam, "mm", gene_name)
            mm_samplelevel_count.append(df.shape[0])
            mm_sample_count += 1 if df.shape[0] > 0 else 0        
        
        return mm_sample_count, mm_samplelevel_count
        
    elif disease == "mgus":
        for sam in all_mgus:
            df = get_gene_df(sam, "mgus", gene_name)
            mgus_samplelevel_count.append(df.shape[0])
            mgus_sample_count += 1 if df.shape[0] > 0 else 0        
        
        return mgus_sample_count, mgus_samplelevel_count

In [5]:
# all_mm[0][:11]

In [6]:
def get_genes(cnr_df):
    genelist = [i.split(',') if ',' in i else [i] for i in cnr_df["gene"].values.tolist() if not "Antitarget" in i]
    genelist = sorted(list(set(list(itertools.chain.from_iterable(genelist)))))
    genelist = [i for i in genelist if not i == '-']
    return genelist

def read_sam_cnvs(sam, disease):
    if disease == "mgus":
        if 'SM' in sam:
            df = pd.read_csv(os.path.join(aiims_mgus_cnv_path, sam, sam+"BM_tumor.call.cns"), sep="\t")
            df = df[df["cn"]!= 2]
            df = df[df["gene"] != '-']
            df = df[~df["chromosome"].str.contains("random")]
            df = df[~df["chromosome"].str.contains("hap")]
            df = df[~df["chromosome"].str.contains("chrUn")].reset_index(drop=True)
        elif "CR" in sam:
            df = pd.read_csv(os.path.join(ega_cnv_path, sam, sam+"-BM_dedup.realigned.call.cns"), sep="\t")
            df = df[df["cn"]!= 2]
            df = df[df["gene"] != '-']
            df = df[~df["chromosome"].str.contains('GL')].reset_index(drop=True)
    elif disease == "mm":
        if 'SM' in sam:
            df = pd.read_csv(os.path.join(aiims_mm_cnv_path, sam, sam+"BM_tumor.call.cns"), sep="\t")
            df = df[df["cn"]!= 2]
            df = df[df["gene"] != '-']
            df = df[~df["chromosome"].str.contains("random")]
            df = df[~df["chromosome"].str.contains("hap")]
            df = df[~df["chromosome"].str.contains("chrUn")].reset_index(drop=True)
        if "MMRF" in sam:
            df = pd.read_csv(os.path.join(mmrf_cnv_path, sam[:11]+"_BM.cns"), sep="\t")            
            
    df['sample'] = [sam for i in range(df.shape[0])]
    return df

def filter_sam_cnvs(sam_df, gen):
    gene_idx = []
    for idx in range(sam_df.shape[0]):
        try:
            string = sam_df.iloc[idx]["gene"]
            genes = sorted(list(set(ast.literal_eval(string))))
        except:
            genes = sorted(list(set(sam_df.iloc[idx]["gene"].split(','))))
        
        if gen in genes:
            gene_idx.append(idx)
    
    return sam_df.iloc[gene_idx]

# def all_samp_gene_cnv(gene_name, disease):
#     df_gene_cnv = pd.DataFrame()
#     if disease == "mm":
#         for k,v in mm_cnv_data['gene_per_sample_dict'].items():
#             if gene_name in v:
#                 # read sample CNVs
#                 df = read_sam_cnvs(k, disease)
#                 # Filter CNV for a given gene
#                 df = filter_sam_cnvs(df, gene_name)
#                 df_gene_cnv = pd.concat([df_gene_cnv, df]).reset_index(drop=True)
#     elif disease == "mgus":
#         for k,v in mgus_cnv_data['gene_per_sample_dict'].items():
#             if gene_name in v:
#                 # read sample CNVs
#                 df = read_sam_cnvs(k, disease)
#                 # Filter CNV for a given gene
#                 df = filter_sam_cnvs(df, gene_name)
#                 df_gene_cnv = pd.concat([df_gene_cnv, df]).reset_index(drop=True)
            
#     no_sample = len(df_gene_cnv["sample"].unique()) if df_gene_cnv.shape[0] > 0 else 0
#     return df_gene_cnv, no_sample


def all_samp_gene_cnv(gene_name, disease):
    df_gene_cnv = pd.DataFrame()
    if disease == "mm":
        for sam in all_mm:
            try:
                # read sample CNVs
                df = read_sam_cnvs(sam, disease)
                # Filter CNV for a given gene
                df = filter_sam_cnvs(df, gene_name)
                df_gene_cnv = pd.concat([df_gene_cnv, df]).reset_index(drop=True)
            except:
                # print(sam)
                continue
    elif disease == "mgus":
        for sam in all_mgus:
            try:
                # read sample CNVs
                df = read_sam_cnvs(sam, disease)
                # Filter CNV for a given gene
                df = filter_sam_cnvs(df, gene_name)
                df_gene_cnv = pd.concat([df_gene_cnv, df]).reset_index(drop=True)
            except:
                continue
            
    return df_gene_cnv, df_gene_cnv["sample"].unique().__len__()

In [7]:
# df = read_sam_cnvs("MMRF_1814_1", "mm")
# print(df.shape)
# # filter_sam_cnvs(df, "RYR2")
# string = df.iloc[0]["gene"]
# genes = sorted(list(set(ast.literal_eval(string))))

In [8]:
# a,b = all_samp_gene_cnv("RYR2","mm")
# a.head()

In [9]:
tgen_df1 = pd.read_csv("/home/vivek/jupyter_notebooks/bio_dgi_extension/SV_literature/mmrf_sv_filtered/mmrf_tgen_filtered500.txt", sep='\t')        
tgen_df1["sv_gene"] = [','.join(set((i+','+j+','+k+','+l+','+m+','+n+','+o).replace('.,','').replace('.','').split(','))) for i,j,k,l,m,n,o in zip(tgen_df1["gene"].tolist(), tgen_df1["GeneInternalBreak1"].tolist(), tgen_df1["GeneInternalBreak2"].tolist(), tgen_df1["GeneUpstreamBreak1"].tolist(), tgen_df1["GeneUpstreamBreak2"].tolist(), tgen_df1["GeneDownstreamBreak1"].tolist(), tgen_df1["GeneDownstreamBreak2"].tolist())]

# Define function to remove trailing comma from a string
def remove_trailing_comma(s):
    return s[:-1] if s.endswith(',') else s

def remove_start_comma(s):
    return s[1:] if s.startswith(',') else s

# Apply function to the "sv_gene" column and update the DataFrame
tgen_df1["sv_gene"] = tgen_df1["sv_gene"].apply(remove_trailing_comma).apply(remove_start_comma)

tgen_df1 = tgen_df1[["SAMPLE", "SvType", "CHROM", "POS", "END", "Gap", "sv_gene"]]
sv_genes = []
for idx in tqdm(range(tgen_df1.shape[0])):
    sam_gene = []
    for gen in tgen_df1.iloc[idx,-1].split(','):
        if gen in top500genes:
            sam_gene.append(gen)
            
    if len(sam_gene) > 1:
        sv_genes.append(sam_gene)
    else:
        sv_genes.append(sam_gene[0])
tgen_df1['genes500'] = sv_genes

delly_tf1 = pd.read_csv("/home/vivek/jupyter_notebooks/bio_dgi_extension/SV_literature/mmrf_sv_filtered/mmrf_delly_filtered500.txt", sep='\t')
delly_tf1["sv_gene"] = [','.join(set((i+','+j+','+k+','+l+','+m).replace('.,','').split(','))) for i,j,k,l,m in zip(delly_tf1["RGENUPS_HUGO"].tolist(), delly_tf1["RGENDNS_HUGO"].tolist(), delly_tf1["LGENUPS_HUGO"].tolist(),delly_tf1["LGENISEC_HUGO"].tolist(),delly_tf1["LGENDNS_HUGO"].tolist())]
delly_tf1 = delly_tf1[["Specimen_ID", "SVTYPE", "CHROM", "POS", "ENDPOSSV", "sv_gene"]]
mmrf_delly_genes500 = []
for idx in range(delly_tf1.shape[0]):
    gs = delly_tf1.iloc[idx,-1].split(',')
    gsn = [i for i in gs if i in top500genes]
    mmrf_delly_genes500.append(','.join(gsn))
    
delly_tf1['genes500'] = mmrf_delly_genes500
delly_tf1["Gap"] = [i-j for i,j in zip(delly_tf1["ENDPOSSV"].tolist(), delly_tf1["POS"].tolist())]
delly_tf1 = delly_tf1.rename({'Specimen_ID': 'SAMPLE', 'SVTYPE': 'SvType','ENDPOSSV':'END'}, axis=1)
delly_tf1 = delly_tf1[['SAMPLE', 'SvType', 'CHROM', 'POS', 'END', 'Gap','sv_gene', 'genes500']]

sv_combined = pd.concat([tgen_df1, delly_tf1])
sv_combined["END"] = [int(i.split(':')[-1]) if isinstance(i, str) else int(i) for i in sv_combined["END"].values.tolist()]
sv_combined.head()

100%|██████████| 5631/5631 [00:00<00:00, 27761.11it/s]


,SAMPLE,SvType,CHROM,POS,END,Gap,sv_gene,genes500
0,MMRF_1020_3_BM,PossibleLargeIndel,5,81571000,98107800,16536800,"RGMB,RPS23",RGMB
1,MMRF_1020_3_BM,PossibleLargeIndel,5,81573800,98109200,16535400,"RGMB,,RPS23",RGMB
2,MMRF_1020_3_BM,PossibleLargeIndel,14,65566200,65592800,26600,MAX,MAX
3,MMRF_1020_3_BM,PossibleLargeIndel,14,65569000,65594200,25200,MAX,MAX
4,MMRF_1020_3_BM,PossibleLargeIndel,14,103230400,103359200,128800,TRAF3,TRAF3


In [10]:
sdict = sv_combined["SvType"].value_counts().to_dict()
print(sdict)

{'PossibleInversion': 4535, 'INV': 793, 'DEL': 768, 'TRA': 573, 'PossibleLargeIndel': 535, 'Translocation': 530, 'DUP': 523, 'PossibleTandemDup': 31}


In [11]:
def filter_gene_svs(df_svs, gene_name):
    df_svs = df_svs.query('genes500 == @gene_name').reset_index(drop=True)
    return df_svs

In [12]:
def snv_top_features(gen_name, disease):
    phylop_median_val, synsnv_total, otherssnv_total, nonsynsnv_total, vaf_nonsyn_snvs, vaf_others_snvs, vaf_syn_snvs = [], [], [], [], [], [], []
    syn_totalratio, nonsyn_totalratio, others_totalratio = [], [], []
    nonsyn_sample_count = 0
    sample_list = all_mm if disease == "mm" else all_mgus
    for sam in sample_list:
        df = pd.read_csv("/home/vivek/jupyter_notebooks/bio_dgi_extension/feature_matrix/genes798feature26/" + sam + ".csv", index_col=0)
        f1 = df[(df.index == gen_name)]["non_syn_phylop_median"].values[0]
        phylop_median_val.append(f1)
        f2 = df[(df.index == gen_name)]["syn_total"].values[0]
        synsnv_total.append(f2)
        f3 = df[(df.index == gen_name)]["non_syn_total"].values[0]
        nonsynsnv_total.append(f3)
        f4 = df[(df.index == gen_name)]["others_total"].values[0]
        otherssnv_total.append(f4)
        # f4 = df[(df.index == gen_name)]["syn_dp_median"].values[0]
        # ad_synsnv_total.append(f4)
        f5 = df[(df.index == gen_name)]["non_syn_vaf_median"].values[0]
        vaf_nonsyn_snvs.append(f5)
        f6 = df[(df.index == gen_name)]["others_vaf_median"].values[0]
        vaf_others_snvs.append(f6)        
        f7 = df[(df.index == gen_name)]["syn_vaf_median"].values[0]
        vaf_syn_snvs.append(f7)
        f8 = f2/(f2+f3+f4)
        f9 = f3/(f2+f3+f4)
        f10 = f4/(f2+f3+f4)
        syn_totalratio.append(f8)
        nonsyn_totalratio.append(f9)
        others_totalratio.append(f10)
        if f3 > 0:
            nonsyn_sample_count += 1
            
        syn_totalratio = [0 if math.isnan(x) else x for x in syn_totalratio]
        nonsyn_totalratio = [0 if math.isnan(x) else x for x in nonsyn_totalratio]
        others_totalratio = [0 if math.isnan(x) else x for x in others_totalratio]
    
    snv_feat_dict = {disease + '_phylop_median_val_median' : np.median(phylop_median_val),
                     disease + '_synsnv_total_median' : np.median(synsnv_total),
                     disease + '_nonsynsnv_total_median' : np.median(nonsynsnv_total),
                     disease + '_otherssnv_total_median' : np.median(otherssnv_total),
                     disease + '_vaf_nonsyn_snvs_median' : np.median(vaf_nonsyn_snvs),
                     disease + '_vaf_syn_snvs_median' : np.median(vaf_syn_snvs),
                     disease + '_vaf_others_snvs_median' : np.median(vaf_others_snvs),
                     disease + '_syn_totalratio_median' : np.median(syn_totalratio),
                     disease + '_nonsyn_totalratio_median' : np.median(nonsyn_totalratio),
                     disease + '_others_totalratio_median' : np.median(others_totalratio),
                     disease + '_others_totalratio_std' : np.std(others_totalratio),
                     disease + '_nonsyn_sample_count' : nonsyn_sample_count}
    
    return snv_feat_dict

In [13]:
def get_snv_features(gen_name):
    mm_snvs, mm_snvs_median = get_gene_snvs_all_samples(gen_name, "mm")
    mgus_snvs, mgus_snvs_median = get_gene_snvs_all_samples(gen_name, "mgus")
    mm_top_feat_dict = snv_top_features(gen_name, "mm")
    mgus_top_feat_dict = snv_top_features(gen_name, "mgus")
    snv_dict = {"mm_sam": mm_snvs, 
                "mm_snvs_median": np.median(mm_snvs_median), 
                "mgus_sam": mgus_snvs, 
                "mgus_snvs_median": np.median(mgus_snvs_median),
                'mm_top_feat_dict': mm_top_feat_dict,
                'mgus_top_feat_dict': mgus_top_feat_dict}
    
    snv_dict1 = {}
    for k,v in snv_dict.items():
        if isinstance(v, dict):
            for k1,v1 in v.items():
                snv_dict1[k1] = v1
        else:
            snv_dict1[k] = v
    return snv_dict1

In [14]:
def get_cnv_features(gen_name):
    mm_cnvs, mm_sam = all_samp_gene_cnv(gen_name, "mm")
    if mm_cnvs.shape[0] > 0:
        mm_cnvs_del = mm_cnvs[mm_cnvs["cn"] == 0]["sample"].unique().shape[0]
        mm_cnvs_loss = mm_cnvs[mm_cnvs["cn"] == 1]["sample"].unique().shape[0]
        mm_cnvs_gain = mm_cnvs[mm_cnvs["cn"] == 3]["sample"].unique().shape[0]
        mm_cnvs_amp = mm_cnvs[mm_cnvs["cn"] > 3]["sample"].unique().shape[0]
        common_end = min(mm_cnvs["end"])
        common_start = max(mm_cnvs["start"])
        common_len = common_end - common_start
        if common_len > 0:
            mm_cnv_overlap = common_len
        else:
            mm_cnv_overlap = 0
    else:
        mm_cnvs_del = 0
        mm_cnvs_loss = 0
        mm_cnvs_gain = 0
        mm_cnvs_amp = 0
        mm_cnv_overlap = 0
        
    mgus_cnvs, mgus_sam = all_samp_gene_cnv(gen_name, "mgus")
    if mgus_cnvs.shape[0] > 0:
        mgus_cnvs_del = mgus_cnvs[mgus_cnvs["cn"] == 0]["sample"].unique().shape[0]
        mgus_cnvs_loss = mgus_cnvs[mgus_cnvs["cn"] == 1]["sample"].unique().shape[0]
        mgus_cnvs_gain = mgus_cnvs[mgus_cnvs["cn"] == 3]["sample"].unique().shape[0]
        mgus_cnvs_amp = mgus_cnvs[mgus_cnvs["cn"] > 3]["sample"].unique().shape[0]
        common_end = min(mgus_cnvs["end"])
        common_start = max(mgus_cnvs["start"])
        common_len = common_end - common_start
        if common_len > 0:
            mgus_cnv_overlap = common_len
        else:
            mgus_cnv_overlap = 0
    else:
        mgus_cnvs_del = 0
        mgus_cnvs_loss = 0
        mgus_cnvs_gain = 0
        mgus_cnvs_amp = 0
        mgus_cnv_overlap = 0
    
    cnv_dict = {'mm_cnvs': mm_sam, 
                'mm_cnvs_del': mm_cnvs_del,
                'mm_cnvs_loss': mm_cnvs_loss,
                'mm_cnvs_gain': mm_cnvs_gain,
                'mm_cnvs_amp': mm_cnvs_amp,
                'mm_cnv_overlap': mm_cnv_overlap,
                'mgus_cnvs': mgus_sam,
                'mgus_cnvs_del': mgus_cnvs_del,
                'mgus_cnvs_loss': mgus_cnvs_loss,
                'mgus_cnvs_gain': mgus_cnvs_gain,
                'mgus_cnvs_amp': mgus_cnvs_amp,
                'mgus_cnv_overlap': mgus_cnv_overlap}
    
    return cnv_dict

In [15]:
def get_sv_features(gen_name):
    df_svs = filter_gene_svs(sv_combined, gen_name)
    sv_dict = df_svs["SvType"].value_counts().to_dict()
    sv_dict['total_svs'] = df_svs.shape[0]
    
    if 'PossibleInversion' in sv_dict.keys() and 'INV' in sv_dict.keys():
        sv_dict['INV'] += sv_dict['PossibleInversion']
        del sv_dict['PossibleInversion']
    elif 'PossibleInversion' in sv_dict.keys() and 'INV' not in sv_dict.keys():
        sv_dict['INV'] = sv_dict['PossibleInversion']
        del sv_dict['PossibleInversion']
        
    if 'TRA' in sv_dict.keys() and 'Translocation' in sv_dict.keys():
        sv_dict['TRA'] += sv_dict['Translocation']
        del sv_dict['Translocation']
    elif 'Translocation' in sv_dict.keys() and 'TRA' not in sv_dict.keys():
        sv_dict['TRA'] = sv_dict['Translocation']
        del sv_dict['Translocation']
        
    if 'DUP' in sv_dict.keys() and 'PossibleTandemDup' in sv_dict.keys():
        sv_dict['DUP'] += sv_dict['PossibleTandemDup']
        del sv_dict['PossibleTandemDup']
    elif 'PossibleTandemDup' in sv_dict.keys() and 'DUP' not in sv_dict.keys():
        sv_dict['DUP'] = sv_dict['PossibleTandemDup']
        del sv_dict['PossibleTandemDup']
        
    sv_dict1 = {}
    sv_dict1["total_svs"] = sv_dict["total_svs"]
    sv_dict1["INV"] = sv_dict["INV"] if "INV" in sv_dict.keys() else 0
    sv_dict1["DUP"] = sv_dict["DUP"] if "DUP" in sv_dict.keys() else 0
    sv_dict1["TRA"] = sv_dict["TRA"] if "TRA" in sv_dict.keys() else 0
    sv_dict1["PossibleLargeIndel"] = sv_dict["PossibleLargeIndel"] if "PossibleLargeIndel" in sv_dict.keys() else 0
    sv_dict1["DEL"] = sv_dict["DEL"] if "DEL" in sv_dict.keys() else 0
    
    return sv_dict1

In [20]:
def get_lof_hap_features(gen_name):
    df = gene_variants[gene_variants['gene_name'] == gen_name]
    lof_hap_dict = {'mm_lof':df["lof_mm"].values[0],
                    'mgus_lof': df["lof_mgus"].values[0],
                    'ghis_score': df["ghis_score"].values[0],
                    'rank': df["rank"].values[0]
                    }    
    return lof_hap_dict

In [21]:
def find_pathway(gen):
    pathways = []
    if gen in antigen_pathway:
        pathways.append("antigen presentation pathway")
    if gen in cyto_pathway:
        pathways.append("cytoskeleton-rearrangement pathway")
    if gen in histone_pathway:
        pathways.append("histone modification pathway")
    if gen in myeloid_pathway:
        pathways.append("myeloid-checkpoints")
    if gen in nk_pathway:
        pathways.append("NK-cell pathway")
        
    return pathways

In [15]:
# gene_df = pd.DataFrame()
# genelist = top500genes[:250]
# rank, sv_chr, cnv_chr = [], [], []

# for gen in tqdm(genelist):
#     rank.append(top500genes.index(gen)+1)
#     try:
#         sv_chr.append(filter_gene_svs(sv_combined,gen).loc[0]["CHROM"])
#     except:
#         sv_chr.append("NA")
        
#     try:
#         df,_ = all_samp_gene_cnv(gen, "mm")
#         cnv_chr.append(df.loc[0]["chromosome"])
#     except:
#         try:
#             df,_ = all_samp_gene_cnv(gen, "mgus")
#             cnv_chr.append(df.loc[0]["chromosome"])
#         except:
#             cnv_chr.append("NA")
    
# gene_df["gene"] = genelist
# gene_df["rank"] = rank
# gene_df["sv_chr"] = sv_chr
# gene_df["cnv_chr"] = cnv_chr
# gene_df.to_excel("gne_cnv_sv_location.xlsx", index=False)
# gene_df.head()

In [16]:
def gene_loc(gen):
    try:
        sv_chr = filter_gene_svs(sv_combined,gen).loc[0]["CHROM"]
    except:
        sv_chr = "NA"
        
    try:
        df,_ = all_samp_gene_cnv(gen, "mm")
        cnv_chr = df.loc[0]["chromosome"]
    except:
        try:
            df,_ = all_samp_gene_cnv(gen, "mgus")
            cnv_chr = df.loc[0]["chromosome"]
        except:
            cnv_chr = "NA"
            
    gene_chr = {"chr_location": cnv_chr if cnv_chr != "NA" else sv_chr}
    # gene_chr = {"sv_chr": sv_chr, "cnv_chr": cnv_chr}
    return gene_chr

In [17]:
def driver_source(gen):
    # driver_dict = {}
    # driver_dict["intogen"] = 1 if gen in driver_list1 else 0
    # driver_dict["list_of_63_driver_genes"] = 1 if gen in driver_list2 else 0
    # driver_dict["Genomic_landscape_and_chronological_reconstruction_of_driver_events_in_multiple"] = 1 if gen in driver_list3 else 0
    
    driver_list = []
    if gen in driver_list1:
        driver_list.append(1)
    else:
        driver_list.append(0)
        
    if gen in driver_list2:
        driver_list.append(2)
    else:
        driver_list.append(0)
    if gen in driver_list3:
        driver_list.append(3)
    else:
        driver_list.append(0)
        
    if gen in driver_list4:
        driver_list.append(4)
    else:
        driver_list.append(0)
        
    if gen in driver_list5:
        driver_list.append(5)
    else:
        driver_list.append(0)
        
    if gen in driver_list6:
        driver_list.append(6)
    else:
        driver_list.append(0)
        
    if gen in driver_list7:
        driver_list.append(7)
    else:
        driver_list.append(0)
    
    
    return driver_list

In [18]:
def gene_category(gen):
    gene_cat = {}
    if gen in cat_a:
        gene_cat["tran_onco_rej"] = "Transformative"
    elif gen in cat_b:
        gene_cat["tran_onco_rej"] = "Rejected"
    elif gen in cat_c:
        gene_cat["tran_onco_rej"] = "Disease_initiating"
        
    # gene_cat["gene34"] = 1 if gen in gene34 else 0
    # gene_cat["gene77"] = 1 if gen in gene77 else 0
    gene_cat["top100"] = 1 if gen in top500genes[:100] else 0
    gene_cat["top250"] = 1 if gen in top500genes[:250] else 0
    return gene_cat

In [ ]:
gene_df = pd.DataFrame()
gen_comm, gen_katz_score = [], []
genelist = gene798
pathways = []
mm_lit_drivers = []

for gen in tqdm(genelist):
    snv_dict = get_snv_features(gen)
    cnv_dict = get_cnv_features(gen)
    sv_dict = get_sv_features(gen)
    lof_hap_dict = get_lof_hap_features(gen)
    a,b = gene_centrality(gen)
    gen_comm.append(a)
    gen_katz_score.append(b)
    pathways.append(find_pathway(gen))
    
    df_snv = pd.DataFrame.from_dict(snv_dict, orient='index', columns=[gen]).T
    df_cnv = pd.DataFrame.from_dict(cnv_dict, orient='index', columns=[gen]).T
    df_sv = pd.DataFrame.from_dict(sv_dict, orient='index', columns=[gen]).T
    df_lof = pd.DataFrame.from_dict(lof_hap_dict, orient='index', columns=[gen]).T
    # df_driver = pd.DataFrame.from_dict(driver_source(gen), orient='index', columns=[gen]).T
    mm_lit_drivers.append(driver_source(gen))
    df_gene_chr = pd.DataFrame.from_dict(gene_loc(gen), orient='index', columns=[gen]).T
    df_gene_cat = pd.DataFrame.from_dict(gene_category(gen), orient='index', columns=[gen]).T
    df_gene_only = pd.concat([df_snv, df_cnv, df_sv, df_lof, df_gene_chr, df_gene_cat], axis=1)
    gene_df = pd.concat([gene_df, df_gene_only])
    
gene_df['pathways'] = pathways
gene_df['mm_driver'] = mm_lit_drivers
# gene_df['rank'] = [int(top500genes.index(gen)+1) for gen in genelist]
gene_df['gen_comm'] = gen_comm
gene_df['gen_katz_score'] = gen_katz_score
gene_df.to_excel("../gene798_features.xlsx", index=True)

In [16]:
gene_df = pd.DataFrame()

for gen in tqdm(gene798):
    
    cnv_dict = get_cnv_features(gen)
    df_cnv = pd.DataFrame.from_dict(cnv_dict, orient='index', columns=[gen]).T
    gene_df = pd.concat([gene_df, df_cnv])
    
gene_df.to_excel("../gene798_cnv_features.xlsx", index=True)

  1%|          | 7/798 [08:21<15:40:54, 71.37s/it]

# Merging of all batches 

In [2]:
df1 = pd.read_excel("gene_features_50.xlsx", index_col=0)
df2 = pd.read_excel("gene_features_100.xlsx", index_col=0)
df3 = pd.read_excel("gene_features_150.xlsx", index_col=0)
df4 = pd.read_excel("gene_features_200.xlsx", index_col=0)
df5 = pd.read_excel("gene_features_250.xlsx", index_col=0)
df6 = pd.read_excel("gene_features_300.xlsx", index_col=0)
df7 = pd.read_excel("gene_features_350.xlsx", index_col=0)
df8 = pd.read_excel("gene_features_400.xlsx", index_col=0)
df9 = pd.read_excel("gene_features_450.xlsx", index_col=0)
df10 = pd.read_excel("gene_features_500.xlsx", index_col=0)
df11 = pd.read_excel("gene_features_550.xlsx", index_col=0)
df12 = pd.read_excel("gene_features_600.xlsx", index_col=0)
df13 = pd.read_excel("gene_features_650.xlsx", index_col=0)
df14 = pd.read_excel("gene_features_700.xlsx", index_col=0)
df15 = pd.read_excel("gene_features_750.xlsx", index_col=0)
df16 = pd.read_excel("gene_features_798.xlsx", index_col=0)
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15, df16], axis=0)
df.to_excel("../gene798_features.xlsx", index=True)
print(df.shape)
df.head()

(798, 58)


,mm_sam,mm_snvs_median,mgus_sam,mgus_snvs_median,mm_phylop_median_val_median,mm_synsnv_total_median,mm_nonsynsnv_total_median,mm_otherssnv_total_median,mm_vaf_nonsyn_snvs_median,mm_vaf_syn_snvs_median,...,ghis_score,rank,chr_location,tran_onco_rej,top100,top250,pathways,mm_driver,gen_comm,gen_katz_score
ABCA1,1001,3.0,52,2,0.0,0,0,3.0,0.0,0.0,...,0.459994,65,chr9,Transformative,1,1,[],"[0, 0, 0, 0, 0, 0, 0]",0.115337,C2
ABCA3,408,0.0,27,0,0.0,0,0,0.0,0.0,0.0,...,0.488285,332,chr16,Disease_initiating,0,0,[],"[0, 0, 0, 0, 0, 0, 0]",NaN,NaN
ABCA7,373,0.0,42,1,0.0,0,0,0.0,0.0,0.0,...,0.520105,441,chr19,Transformative,0,0,[],"[0, 0, 0, 0, 0, 0, 0]",0.019731,C4
ABL2,776,1.0,30,0,0.0,0,0,1.0,0.0,0.0,...,0.533039,175,chr1,Transformative,0,1,[],"[0, 0, 0, 0, 0, 0, 0]",0.086526,C2
ACACB,496,0.0,40,1,0.0,0,0,0.0,0.0,0.0,...,0.565962,339,chr12,Transformative,0,0,[],"[0, 0, 0, 0, 0, 0, 0]",0.079910,C2


In [6]:
hgtable = pd.read_csv("/home/vivek/refs/ucsc_data/hgTables.csv", header = 0)
hgtable_genename = pd.read_csv("/home/vivek/refs/ucsc_data/hgTables_gene_name.txt", sep="\t", header=0)
hgtable_genename = hgtable_genename[hgtable_genename["geneSymbol"].isin(gene798)]

def fetch_geneid(s):
    return s.split(";")[1].split(" ")[-1].replace('"', '')

def fetch_genename(s):
    return hgtable_genename[hgtable_genename["#kgID"] == s]["geneSymbol"].values[0]

def fetch_transcripts(gene_name):
    return hgtable_genename[hgtable_genename["geneSymbol"] == gene_name]["#kgID"].values.tolist()

In [18]:
hgtable[hgtable["alignID"] == fetch_transcripts("KRAS")[0]]["chrom"].values[0]

'chr12'

In [22]:
gene_chr_loc = pd.DataFrame()
gene_chr_loc["gene"] = gene798

chr_loc = []

for gen in tqdm(gene798):
    try:
        chr_loc.append(hgtable[hgtable["alignID"] == fetch_transcripts(gen)[0]]["chrom"].values[0])
    except:
        print(gen, fetch_transcripts(gen))
        chr_loc.append("NA")
        
gene_chr_loc["chr_loc"] = chr_loc
gene_chr_loc.to_excel("../gene_chr_loc.xlsx", index=False)
gene_chr_loc.head()

  4%|▍         | 32/798 [00:00<00:05, 136.88it/s]

 15%|█▌        | 121/798 [00:00<00:04, 144.48it/s]

CMTR2 []
CXorf67 []


 57%|█████▋    | 452/798 [00:03<00:02, 163.06it/s]

NPIPA5 []
NPIPB15 []


...

SPATA31A1 []
SUGCT []
TBC1D3F []


 94%|█████████▍| 751/798 [00:05<00:00, 156.81it/s]

WASH4P []


100%|██████████| 798/798 [00:05<00:00, 147.88it/s]


,gene,chr_loc
0,ABCA1,chr9
1,ABCA3,chr16
2,ABCA7,chr19
3,ABL2,chr1
4,ACACB,chr12
